<a href="https://colab.research.google.com/github/alonlavian/AlmaZohar/blob/master/FaceCrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trying to enlarge my data set using Google vision API

For the following to work you'll have to setup:
*   service account credentials - https://cloud.google.com/docs/authentication/production#obtaining_and_providing_service_account_credentials_manually
*   google billing  - https://support.google.com/googleapi/answer/6158867?hl=en




1.   Install google cloud vision

In [7]:
!pip install --upgrade google-cloud-vision


    100% |████████████████████████████████| 389kB 25.5MB/s 



2.   Mount Drive

In [5]:
import os 
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


3. Add vision application credentilas to env.

    https://cloud.google.com/docs/authentication/production#obtaining_and_providing_service_account_credentials_manually

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/ML/Misc/vision_service_account.json"

4. Cropping code
    https://cloud.google.com/vision/docs/crop-hints


In [0]:
import argparse
import io

from google.auth import compute_engine
from google.cloud import vision
from google.cloud.vision import types
from PIL import Image, ImageDraw


def get_crop_hint(path):
    """Detect crop hints on a single image and return the first result."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    crop_hints_params = types.CropHintsParams(aspect_ratios=[1.77])
    image_context = types.ImageContext(crop_hints_params=crop_hints_params)

    response = client.crop_hints(image=image, image_context=image_context)
    hints = response.crop_hints_annotation.crop_hints

    # Get bounds for the first crop hint using an aspect ratio of 1.77.
    vertices = hints[0].bounding_poly.vertices

    return vertices


# def draw_hint(image_file):
#     """Draw a border around the image using the hints in the vector list."""
#     vects = get_crop_hint(image_file)

#     im = Image.open(image_file)
#     draw = ImageDraw.Draw(im)
#     draw.polygon([
#         vects[0].x, vects[0].y,
#         vects[1].x, vects[1].y,
#         vects[2].x, vects[2].y,
#         vects[3].x, vects[3].y], None, 'red')
#     im.save('output-hint.jpg', 'JPEG')
#     print('Saved new image to output-crop.jpg')

#         crop(path,filename, cropped_photos_dir)

def crop_to_hint(path,filename, cropped_photos_dir):
    """Crop the image using the hints in the vector list."""
    image_file = os.path.join(path,os.fsencode(filename))
    vects = get_crop_hint(image_file)

    im = Image.open(image_file)
    im2 = im.crop([vects[0].x, vects[0].y,
                  vects[2].x - 1, vects[2].y - 1])
    
    cropped_image_name = "cropped_" + filename
    cropped_image_path = os.path.join(cropped_photos_dir,os.fsencode(cropped_image_name))
    im2.save(cropped_image_path, 'JPEG')
    print('Saved new image to output-crop.jpg')


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('image_file', help='The image you\'d like to crop.')
#     parser.add_argument('mode', help='Set to "crop" or "draw".')
#     args = parser.parse_args()

#     parser = argparse.ArgumentParser()

#     if args.mode == 'crop':
#         crop_to_hint(args.image_file)
#     elif args.mode == 'draw':
#         draw_hint(args.image_file)

Try to crop

In [0]:




crop_to_hint('/content/drive/My Drive/Google Photos/2019/04/IMG_20190402_075133.jpg')

Saved new image to output-crop.jpg


In [0]:
import random,os

original_photos_dir = os.fsencode('/content/drive/My Drive/Google Photos/2019')
cropped_photos_dir  = os.fsencode('/content/drive/My Drive/ML/AlmaZoharCrop/CropMix')
if not os.path.exists(cropped_photos_dir):
    os.makedirs(cropped_photos_dir)
    
    
for path, dirs, files in os.walk(original_photos_dir):
  if files:
    rand_files = (random.sample(files,1))
    for file in rand_files:
      filename = os.fsdecode(file)
      if filename.endswith(".jpg"):
#         image_to_crop = os.path.join(path,file_to_crop)
        print("Cropping ", filename)
        crop_to_hint(path,filename, cropped_photos_dir)
        

Cropping  IMG_20190131_175203.jpg
Saved new image to output-crop.jpg
Cropping  IMG_20190208_132441.jpg
Saved new image to output-crop.jpg
Cropping  IMG_20190322_125150.jpg
Saved new image to output-crop.jpg
Cropping  IMG_20190409_130236.jpg
Saved new image to output-crop.jpg


Detecting the faces
https://cloud.google.com/vision/docs/detecting-faces

In [10]:
def detect_face(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = types.Image(content=content)

    return client.face_detection(image=image, max_results=max_results).face_annotations
  
  
  
  
  
with open('IMG_20190218_061506.jpg', 'rb') as image:
        faces = detect_face(image, 2)
        print('Found {} face{}'.format(
            len(faces), '' if len(faces) == 1 else 's'))
      
print(faces)
  


Found 1 face
[bounding_poly {
  vertices {
    x: 891
    y: 561
  }
  vertices {
    x: 1280
    y: 561
  }
  vertices {
    x: 1280
    y: 1012
  }
  vertices {
    x: 891
    y: 1012
  }
}
fd_bounding_poly {
  vertices {
    x: 953
    y: 680
  }
  vertices {
    x: 1230
    y: 680
  }
  vertices {
    x: 1230
    y: 957
  }
  vertices {
    x: 953
    y: 957
  }
}
landmarks {
  type: LEFT_EYE
  position {
    x: 1019.197265625
    y: 765.2638549804688
    z: -0.001612532534636557
  }
}
landmarks {
  type: RIGHT_EYE
  position {
    x: 1143.3802490234375
    y: 768.7728881835938
    z: -8.067333221435547
  }
}
landmarks {
  type: LEFT_OF_LEFT_EYEBROW
  position {
    x: 984.7256469726562
    y: 735.4658813476562
    z: 13.025546073913574
  }
}
landmarks {
  type: RIGHT_OF_LEFT_EYEBROW
  position {
    x: 1052.4207763671875
    y: 738.0965576171875
    z: -25.593000411987305
  }
}
landmarks {
  type: LEFT_OF_RIGHT_EYEBROW
  position {
    x: 1109.29150390625
    y: 740.6908569335938
